# Module 3: Model Training
**This notebook uses the feature set extracted by `module-2` to create a XGBoost based machine learning model for binary classification**

**Note:** Please set kernel to `Python 3 (Data Science)` and select instance to `ml.t3.medium`

---

## Contents

1. [Background](#Background)
1. [Setup](#Setup)
1. [Load transformed feature set](#Load-transformed-feature-set)
1. [Split data](#Split-data)
1. [Train a model using SageMaker built-in XgBoost algorithm](#Train-a-model-using-SageMaker-built-in-XgBoost-algorithm)
1. [Real time inference using the deployed endpoint](#Real-time-inference-using-the-deployed-endpoint)

# Background

In this notebook, we demonstrate how to use the feature set derived in `Module-2` and create a machine learning model for predicting whether a customer will reorder a product or not based on historical records. Given the problem type is supervised binary classification, we will use a SageMaker built-in algorithm XGBoost to design this classifier. Once the model is trained, we will also deploy the trained model as a SageMaker endpoint for real-time inference.

# Setup

In [3]:
from sagemaker.serializers import CSVSerializer
from sagemaker.inputs import TrainingInput
from sagemaker.predictor import Predictor
from sagemaker import get_execution_role
import pandas as pd
import numpy as np
import sagemaker
import logging
import boto3
import json
import os

In [4]:
logger = logging.getLogger('__name__')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

#### Essentials

In [5]:
sagemaker_execution_role = get_execution_role()
logger.info(f'Role = {sagemaker_execution_role}')
session = boto3.Session()
sagemaker_session = sagemaker.Session()
default_bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker-featurestore-workshop'
s3 = session.resource('s3')

Role = arn:aws:iam::570358149193:role/service-role/AmazonSageMaker-ExecutionRole-20210506T162928
Role = arn:aws:iam::570358149193:role/service-role/AmazonSageMaker-ExecutionRole-20210506T162928


# Load transformed feature set

In [6]:
df = pd.read_csv('../data/train/transformed.csv')
df.head(5)

,sex,is_married,age_18-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-plus,n_days_active,category_baby_food_formula,...,category_ice_cream_ice,category_juice_nectars,category_packaged_cheese,category_refrigerated,category_soup_broth_bouillon,category_spices_seasonings,category_tea,category_vitamins_supplements,category_yogurt,is_reordered
0,0,1,0,0,0,0,0,1,0.767123,0,...,0,1,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,1,0.767123,0,...,0,1,0,0,0,0,0,0,0,0
2,1,0,1,0,0,0,0,0,0.593151,0,...,0,0,0,0,0,0,1,0,0,0
3,1,0,1,0,0,0,0,0,0.593151,0,...,0,0,1,0,0,0,0,0,0,1
4,1,0,1,0,0,0,0,0,0.593151,0,...,0,0,1,0,0,0,0,0,0,1


In [7]:
df.shape

(328131, 30)

Move column `is_redordered` to be the first column since our training algorithm `XGBoost` expects the target column to be the first column.

In [8]:
first_column = df.pop('is_reordered')
df.insert(0, 'is_reordered', first_column)
df.head()

,is_reordered,sex,is_married,age_18-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-plus,n_days_active,...,category_hair_care,category_ice_cream_ice,category_juice_nectars,category_packaged_cheese,category_refrigerated,category_soup_broth_bouillon,category_spices_seasonings,category_tea,category_vitamins_supplements,category_yogurt
0,0,0,1,0,0,0,0,0,1,0.767123,...,0,0,1,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,1,0.767123,...,0,0,1,0,0,0,0,0,0,0
2,0,1,0,1,0,0,0,0,0,0.593151,...,0,0,0,0,0,0,0,1,0,0
3,1,1,0,1,0,0,0,0,0,0.593151,...,0,0,0,1,0,0,0,0,0,0
4,1,1,0,1,0,0,0,0,0,0.593151,...,0,0,0,1,0,0,0,0,0,0


# Split data

We will shuffle the whole dataset first (df.sample(frac=1, random_state=123)) and then split our data set into the following parts:

* 70% - train set,
* 20% - validation set,
* 10% - test set

**Note:**  In the code below, the first element denotes size for train (0.7 = 70%), second element denotes size for test (1-0.9 = 0.1 = 10%) and difference between the two denotes size for validation(1 - [0.7+0.1] = 0.2 = 20%).

In [9]:
train_df, validation_df, test_df = np.split(df.sample(frac=1, random_state=123), [int(.7*len(df)), int(.9*len(df))])

In [10]:
train_df.shape

(229691, 30)

In [11]:
validation_df.shape

(65626, 30)

In [12]:
test_df.shape

(32814, 30)

Save split datasets to local

In [13]:
train_df.to_csv('../data/train/train.csv', index=False)
validation_df.to_csv('../data/validation/validation.csv', index=False)
test_df.to_csv('../data/test/test.csv', index=False)

Copy datasets to S3 from local

In [14]:
s3.Bucket(default_bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('../data/train/train.csv')
s3.Bucket(default_bucket).Object(os.path.join(prefix, 'validation/validation.csv')).upload_file('../data/validation/validation.csv')
s3.Bucket(default_bucket).Object(os.path.join(prefix, 'test/test.csv')).upload_file('../data/test/test.csv')

Create Pointers to the uploaded files

In [15]:
train_set_location = 's3://{}/{}/train/'.format(default_bucket, prefix)
validation_set_location = 's3://{}/{}/validation/'.format(default_bucket, prefix)
test_set_location = 's3://{}/{}/test/'.format(default_bucket, prefix)

In [16]:
train_set_pointer = TrainingInput(s3_data=train_set_location, content_type='csv')
validation_set_pointer = TrainingInput(s3_data=validation_set_location, content_type='csv')
test_set_pointer = TrainingInput(s3_data=test_set_location, content_type='csv')

In [17]:
print(json.dumps(train_set_pointer.__dict__, indent=2))

{
  "config": {
    "DataSource": {
      "S3DataSource": {
        "S3DataType": "S3Prefix",
        "S3Uri": "s3://sagemaker-ap-southeast-1-570358149193/sagemaker-featurestore-workshop/train/",
        "S3DataDistributionType": "FullyReplicated"
      }
    },
    "ContentType": "csv"
  }
}


# Train a model using SageMaker built-in XgBoost algorithm

In [18]:
container_uri = sagemaker.image_uris.retrieve(region=session.region_name, 
                                              framework='xgboost', 
                                              version='1.0-1', 
                                              image_scope='training')

In [19]:
xgb = sagemaker.estimator.Estimator(image_uri=container_uri,
                                    role=sagemaker_execution_role, 
                                    instance_count=2, 
                                    instance_type='ml.m5.xlarge',
                                    output_path='s3://{}/{}/model-artifacts'.format(default_bucket, prefix),
                                    sagemaker_session=sagemaker_session,
                                    base_job_name='reorder-classifier')

xgb.set_hyperparameters(objective='binary:logistic',
                        num_round=100)

In [20]:
xgb.fit({'train': train_set_pointer, 'validation': validation_set_pointer})

2021-10-26 00:47:45 Starting - Starting the training job...
2021-10-26 00:48:08 Starting - Launching requested ML instancesProfilerReport-1635209264: InProgress
...
2021-10-26 00:48:38 Starting - Preparing the instances for training.........
2021-10-26 00:50:08 Downloading - Downloading input data...
2021-10-26 00:50:35 Training - Training image download completed. Training in progress.INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
[00:50:37] 229692x29 matrix with 6661068 entries loaded from /opt/ml/input/data/train?format

In [21]:
xgb_model_data = xgb.model_data
%store xgb_model_data

Stored 'xgb_model_data' (str)


#### Host the trained XGBoost model as a SageMaker Endpoint

**Note:** The deployment usually takes ~10 mins - good time to take a coffee break :)

In [ ]:
xgb_predictor = xgb.deploy(initial_instance_count=2,
                           instance_type='ml.m5.xlarge')

# Real time inference using the deployed endpoint

In [ ]:
csv_serializer = CSVSerializer()
endpoint_name = xgb_predictor.endpoint_name
%store endpoint_name
predictor = Predictor(endpoint_name=endpoint_name, 
                      serializer=csv_serializer)

In [ ]:
test_df = pd.read_csv('../data/test/test.csv')
record = test_df.sample(1)
record

In [ ]:
X = record.values[0]
payload = X[1:]
payload

In [ ]:
%%time

predicted_class_prob = predictor.predict(payload).decode('utf-8')
if float(predicted_class_prob) < 0.5:
    logger.info('Prediction (y) = Will not reorder')
else:
    logger.info('Prediction (y) = Will reorder')